In [ ]:
%pylab inline

# Author's gender identification

## 0. Read dataset 

In [5]:
import xlrd

In [6]:
path = 'blog-gender-dataset.xlsx'

workbook = xlrd.open_workbook(path)
worksheet = workbook.sheet_by_index(0)

# Change this depending on how many header rows are present
# Set to 0 if you want to include the header data.
offset = 0

rows = []
for i, row in enumerate(range(worksheet.nrows)):
    if i <= offset:  # (Optionally) skip headers
        continue
    r = []
    for j, col in enumerate(range(worksheet.ncols)):
        r.append(worksheet.cell_value(i, j))
    rows.append(r[:2])

In [7]:
print('Got {} rows'.format(len(rows)))

Got 3231 rows


In [8]:
x_raw = []
y = []

for i in rows:
    x_raw.append(i[0])
    y.append(i[1])

In [9]:
import re
import nltk
from random import shuffle

In [10]:
def get_words(text, stop=False):
    raw_text =  re.split(' ',text)
    unwanted = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~£'
    raw_text2 = ''
    for k in raw_text:
        # Ignore urls
        if k.startswith('http') or k.startswith('http'):
            continue
        elif k.startswith('@'):     
            continue
        elif k.startswith('#'):
            continue
        else:
            raw_text2 += ' '
            raw_text2 += k
    words_raw = nltk.word_tokenize(raw_text2.lower())
    words = [w for w in words_raw if w not in unwanted]
    if stop: 
        words = [w for w in words if w not in stopwords.words('english')]
    return words

In [11]:
%%time
sent_pruning = 400
model_dict = {}
x_encoded = []
id = 1
for sent in x_raw:
    words = []
    for word in get_words(sent):
        if word not in model_dict.keys():
            model_dict[word] = id
            id += 1
        words.append(model_dict[word])
    x_encoded.append(words)

CPU times: user 11.9 s, sys: 16 ms, total: 11.9 s
Wall time: 11.9 s


In [12]:
top_words = len(model_dict)+1
print(top_words)

66259


# 1. Feature transform

In [6]:
import nltk
from nltk.corpus import stopwords
import pickle as pl
import re
import seaborn as sns

In [18]:
def word_features(text):
    raw_text =  re.split(' ',text)
    unwanted = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~£'
    raw_text2 = ''
    for k in raw_text:
        # Ignore urls
        if k.startswith('http') or k.startswith('http'):
            continue
        elif k.startswith('@'):     
            continue
        elif k.startswith('#'):
            continue
        else:
            raw_text2 += ' '
            raw_text2 += k
    words_raw = nltk.word_tokenize(raw_text2.lower())
    words = [w for w in words_raw if w not in unwanted]
    feat = {}
    for w in words:
        if w in feat.keys():
            feat[w] += 1
        else:
            feat[w] = 1
    return feat

In [178]:
# split to test and train

from random import shuffle
# Given list1 and list2
x_rand = []
y_rand = []
index_shuf = list(range(len(x_raw)))
shuffle(index_shuf)

for i in index_shuf:
    x_rand.append(x_raw[i])
    y_rand.append(y[i])

split_stamp = int(0.8 * len(x_rand))  
x_train = x_rand[:split_stamp]
y_train = y_rand[:split_stamp]

x_test = x_rand[split_stamp:]
y_test = y_rand[split_stamp:]

    

In [8]:
from random import choice
import sklearn as sk

In [ ]:
labels = ['M', 'F']
y_pred = [choice(labels) for i in range(len(x_test))]
print("Random guess score: ")
print(sk.metrics.accuracy_score(y_test, y_pred))

In [ ]:
def word2ngrams(text, n=3, exact=True):
    """ Convert sentence into character ngrams. """
    yield [text[i:i+n] for i in range(len(text)-n+1)]

In [31]:
def features(words, n=2):
    feat = {}
    for w in words:
        bi_grams = word2ngrams(w, n=n)
        for inst in bi_grams:
            for bg in inst:
                if bg in feat.keys():
                    feat[bg] +=  1
                else:
                    feat[bg] = 1

    return feat
    

In [29]:
def get_features(text, n_grams = 3, mode ='incl'):
    '''
     Transform sentence into feature vector 
     text: Sentence 
     n_grams: charachter ngram used
     mode: {'incl', None} If incl : Feature vector will conatin ngrams from 1 to n 
    '''
    raw_text =  re.split(' ',text)
    unwanted = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~£'
    raw_text2 = ''
    for k in raw_text:
        # Ignore urls
        if k.startswith('http') or k.startswith('http'):
            continue
        elif k.startswith('@'):     
            continue
        elif k.startswith('#'):
            continue
        else:
            raw_text2 += ' '
            raw_text2 += k
    words_raw = nltk.word_tokenize(raw_text2.lower())
    words = [w for w in words_raw if w not in unwanted]
    feat = word_features(text)
    feat.update(features(words, n=n_grams))
    #words_set = set(words)
    for lang in labels:
        feat[lang+'_stop'] = len(set(words).intersection(set(stopwords.words(lang))))
    if mode == 'incl':
        for i in range(1, n_grams):
            feat.update(features(words, n=i))
    elif mode == 'incl_except': 
         for i in range(2, n_grams):
            feat.update(features(words, n=i))
    return feat

In [2]:
def word_features(terms, inv_terms, text):
    raw_text =  re.split(' ',text)
    unwanted = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~£'
    raw_text2 = ''
    for k in raw_text:
        # Ignore urls
        if k.startswith('http') or k.startswith('http'):
            continue
        elif k.startswith('@'):     
            continue
        elif k.startswith('#'):
            continue
        else:
            raw_text2 += ' '
            raw_text2 += k
    words_raw = nltk.word_tokenize(raw_text2.lower())
    words = [w for w in words_raw if w not in unwanted]
    feat = {}
    for w in words:
        if w in feat.keys():
            feat[w] += 1
        else:
            feat[w] = 1
    return feat

In [ ]:
print(len(x_raw))

In [23]:
from nltk.corpus import stopwords

In [152]:
from nltk import ngrams
def split_into_tokens(message):
    return message.split()
def split_into_bigram_tokens(message, n=5):
    res = []
    for i in range(1, n+1):
        res += [x for x in ngrams(message.split(),i)]
    return res

# 2. Train and classify

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.grid_search import GridSearchCV

/home/bulat/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/bulat/.local/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [32]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=get_words)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('nb', MultinomialNB(alpha=0.6)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

TypeError: get_words() missing 1 required positional argument: 'text'

In [30]:
%%time
pipeline.fit(x_train, y_train)

CPU times: user 8.98 s, sys: 20 ms, total: 9 s
Wall time: 9 s


Pipeline(steps=[('bow', CountVectorizer(analyzer=<function get_words at 0x7f6108fd07b8>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,...linear_tf=False, use_idf=True)), ('nb', MultinomialNB(alpha=0.6, class_prior=None, fit_prior=True))])

In [31]:
%%time
pipeline.score(x_test, y_test)

CPU times: user 3.67 s, sys: 0 ns, total: 3.67 s
Wall time: 3.67 s


0.64948453608247425

In [171]:
from sklearn.svm import SVC

In [221]:
pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words='english', lowercase=True, ngram_range=(1,1), analyzer='word')),  # strings to token integer counts
    ('tfidf', TfidfTransformer(smooth_idf=True)),  # integer counts to weighted TF-IDF scores
    ('nb', MultinomialNB(alpha=0.01)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [222]:
%%time
pipeline.fit(x_train, y_train)

CPU times: user 992 ms, sys: 8 ms, total: 1e+03 ms
Wall time: 1e+03 ms


Pipeline(steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...inear_tf=False, use_idf=True)), ('nb', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))])

In [223]:
%%time
pipeline.score(x_test, y_test)

CPU times: user 200 ms, sys: 0 ns, total: 200 ms
Wall time: 201 ms


0.65069551777434309

In [65]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier

In [125]:
pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words='english', lowercase=True, ngram_range=(1,1))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(smooth_idf=True)),  # integer counts to weighted TF-IDF scores
    ('rf', ),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [126]:
%%time
pipeline.fit(x_train, y_train)

CPU times: user 40.9 s, sys: 120 ms, total: 41 s
Wall time: 3.54 s


Pipeline(steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...ators=800, n_jobs=20, oob_score=False,
            random_state=True, verbose=0, warm_start=False))])

In [127]:
%%time
pipeline.score(x_test, y_test)

CPU times: user 2.14 s, sys: 164 ms, total: 2.3 s
Wall time: 789 ms


0.62371134020618557

In [116]:
pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words='english', lowercase=True, ngram_range=(1,1))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(smooth_idf=True)),  # integer counts to weighted TF-IDF scores
    ('rf', GradientBoostingClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [117]:
%%time
pipeline.fit(x_train, y_train)

CPU times: user 1min 3s, sys: 4 ms, total: 1min 3s
Wall time: 1min 3s


Pipeline(steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        s...=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))])

In [119]:
%%time
pipeline.score(x_test, y_test)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

## Gensim models

In [366]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence, TaggedDocument, TaggedLineDocument
from gensim.models import Doc2Vec

# numpy
import numpy

# random
from random import shuffle

# classifier
from sklearn.linear_model import LogisticRegression

In [369]:
class LabeledLineSentence(object):
    def __init__(self):
        self.sentences = []
                
    def feed(self, x, y, lab):
        for i in range(len(x)):
            words = get_words(x[i])
            label = [i]
            self.sentences.append(TaggedDocument(words, label))
    
    def __iter__(self):
        for w, l in self.sentences:
            yield TaggedDocument(w, l)
    
    def to_array(self):
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [370]:
# 
sentences = LabeledLineSentence()
sentences.feed(x_train, y_train, 'train')
sentences.feed(x_test, y_test, 'test')

In [294]:
model = Doc2Vec(size=100, workers=20, min_count=1, window=10)

In [372]:
model.build_vocab(sentences)

RuntimeError: must sort before initializing vectors/weights

In [296]:
for epoch in range(10):
    model.train(sentences.sentences_perm())
    print('epoch '+str(epoch))

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [297]:
w, l = sentences.to_array()[2]

In [305]:
model.most_similar('train')

[('room', 0.5349263548851013),
 ('bus', 0.5344296097755432),
 ('bathroom', 0.5092256665229797),
 ('anti-christ', 0.4951900839805603),
 ('walk', 0.4834766387939453),
 ('minimum', 0.48019909858703613),
 ('window', 0.47659388184547424),
 ('gym', 0.47524479031562805),
 ('plan', 0.4721883237361908),
 ('chapel', 0.47153937816619873)]

In [425]:

class LabeledLineSentence(object):
    def __init__(self, prefix, sources):
        self.sources = sources
        self.prefix = prefix
        k1 = self.to_array()
        
    
    def __iter__(self):
        for item_no, line in enumerate(self.sources):
            yield LabeledSentence(get_words(line[0]), [self.prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for item_no, line in enumerate(self.sources):
            self.sentences.append(LabeledSentence(get_words(line[0]), [self.prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [545]:
%%time
sentences = LabeledLineSentence('gen',rows)

CPU times: user 13.5 s, sys: 68 ms, total: 13.6 s
Wall time: 13.6 s


In [625]:
%%time
model = Doc2Vec(min_count=1, window=8, size=350, sample=1e-4, negative=5, workers=20)
model.build_vocab(sentences)

CPU times: user 15.8 s, sys: 108 ms, total: 15.9 s
Wall time: 15.9 s


In [626]:
%%time
for epoch in range(15):
    print(epoch)
    model.train(sentences.sentences_perm())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
CPU times: user 20min 38s, sys: 15.1 s, total: 20min 53s
Wall time: 2min 32s


In [9]:
%%time
idx = list(range(len(rows)))
shuffle(idx)
stamp = int(0.8*len(idx))

x_rand = []
y_rand = []
for i in idx:
    x_rand.append(model.docvecs['gen_{}'.format(i)])
    y_rand.append(y[i])

split_stamp = int(0.8 * len(x_rand))  
x_train = x_rand[:split_stamp]
y_train = y_rand[:split_stamp]

x_test = x_rand[split_stamp:]
y_test = y_rand[split_stamp:]


y_train_norm = []
for i in range(len(y_train)):
    if 'F' in y_train[i]:
        y_train_norm.append(0)
    else:
        y_train_norm.append(1)
        
y_test_norm = []
for i in range(len(y_test)):
    if 'F' in y_test[i]:
        y_test_norm.append(0)
    else:
        y_test_norm.append(1)

NameError: name 'model' is not defined

In [624]:
%%time
svm = SVC()
parameters = {
    'degree':[3,4,5]
}

clf = GridSearchCV(svm, parameters, cv=3, n_jobs=20)
clf.fit(x_train, y_train_norm)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Best score: %0.3f" % clf.best_score_)

print("Test score")
print(clf.score(x_test, y_test_norm))


Best parameters set found on development set:

{'degree': 3}

Best score: 0.690
Test score
0.681607418856
CPU times: user 4.27 s, sys: 1.53 s, total: 5.8 s
Wall time: 8.4 s


In [618]:
%%time
rf1 = RandomForestClassifier( n_jobs=20, max_depth=None, random_state=False)
parameters = {
    'n_estimators':list(range(100, 1000,100))
}

clf = GridSearchCV(rf1, parameters, cv=3, n_jobs=5)
clf.fit(x_train, y_train_norm)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Best score: %0.3f" % clf.best_score_)
print("Test score")
print(clf.score(x_test, y_test_norm))

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<

In [132]:
most_informative_feature_for_class(word_vectorizer, mnb, 'M')

M life -8.1486015601
M really -8.14416341693
M think -8.11779365425
M new -8.10387104746
M know -8.10296767941
M good -8.04139473373
M people -7.91682594927
M time -7.78305124831
M just -7.75619653777
M like -7.68156810442


In [133]:
most_informative_feature_for_class(word_vectorizer, mnb, 'F')

F life -8.02726357489
F don -7.97137694716
F ve -7.95612851579
F day -7.94131233097
F love -7.92326336388
F really -7.83579676169
F know -7.7844598684
F time -7.66261104082
F like -7.62972744295
F just -7.61585286709
